# Predicting Hospital Stock Requirements using SARIMA

Importing the required modules for this project

In [4]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import matplotlib.pyplot as plt
%matplotlib inline

## Load and Prepare Data

Loading the dataset containing historical monthly consumption data for various medical items over the past three years. We will reshape this data to facilitate time series analysis.

In [6]:
# Load and prepare data
data = pd.read_csv('data.csv')

# Reshape data
monthly_data = pd.melt(data, id_vars=['ITEM DESCRIPTION'],
                       value_vars=[col for col in data.columns if 'JANUARY' in col or 'FEBRUARY' in col or 'MARCH' in col or 'APRIL' in col or 'MAY' in col or 'JUNE' in col or 'JULY' in col or 'AUGUST' in col or 'SEPTEMBER' in col or 'OCTOBER' in col or 'NOVEMBER' in col or 'DECEMBER' in col],
                       var_name='Month_Year', value_name='Consumption')

# Extract year and month
monthly_data['Year'] = monthly_data['Month_Year'].str.split('_').str[1]
monthly_data['Month'] = monthly_data['Month_Year'].str.split('_').str[0]

# Create a date column
monthly_data['Date'] = pd.to_datetime(monthly_data['Month'] + ' ' + monthly_data['Year'], format='%B %Y')

# Drop unnecessary columns
monthly_data = monthly_data.drop(columns=['Month_Year'])

# Sort by date
monthly_data = monthly_data.sort_values(by='Date')

# Set Date as index
monthly_data.set_index('Date', inplace=True)

# Display the first few rows of the reshaped data
monthly_data.head()

,ITEM DESCRIPTION,Consumption,Year,Month
Date,,,,
2019-01-01,18 - Reactive Protein (CRP) - 300t - Dt Slides...,4,2019,JANUARY
2019-01-01,Ringer Lactate - Compound Sodium Lactate Injec...,119,2019,JANUARY
2019-01-01,Ring File 79,33,2019,JANUARY
2019-01-01,Ribbon Gauze Non-Sterile (VAGINAL Pack) 5m x 5...,6,2019,JANUARY
2019-01-01,Ribbon Gauze Non-Sterile (ENT Pack) 5m x 1.25c...,0,2019,JANUARY


## Forecast with SARIMA

For each item in the dataset, we will:
- Extract the monthly consumption data.
- Fit a SARIMA model to this time series data.
- Forecast the consumption for the next 12 months.
- Store the forecast results.

In [7]:
# Prepare for each item
items = monthly_data['ITEM DESCRIPTION'].unique()

future_predictions_all_items = []

for item in items:
    # Filter data for the current item
    item_data = monthly_data[monthly_data['ITEM DESCRIPTION'] == item].copy()

    # Fit SARIMA model
    try:
        model = SARIMAX(item_data['Consumption'], order=(1, 1, 1), seasonal_order=(0, 1, 1, 12))
        sarima_model = model.fit(disp=False)

        # Forecast next 12 months
        future_dates = pd.date_range(start=item_data.index.max() + pd.DateOffset(months=1), periods=12, freq='MS')
        forecast = sarima_model.get_forecast(steps=12)
        forecast_values = forecast.predicted_mean

        # Create DataFrame for future predictions
        future_df = pd.DataFrame({'Date': future_dates, 'Item': item, 'Predicted_Consumption': forecast_values})
        future_predictions_all_items.append(future_df)

    except Exception as e:
        print(f"Error processing item {item}: {e}")

# Combine all future predictions into a single DataFrame
final_predictions = pd.concat(future_predictions_all_items)

c:\Users\Ankhs3ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\Ankhs3ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Users\Ankhs3ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:1009: UserWarning: Non-invertible starting seasonal moving average Using zeros as starting parameters.
  warn('Non-invertible starting seasonal moving average'
c:\Users\Ankhs3ram\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
c:\Us

### Save predictions to CSV

In [ ]:
# Save predictions to CSV
final_predictions.to_csv('future_predictions.csv', index=False)
print("Predictions saved to 'future_predictions.csv'")

# Display the first few rows of the predictions
final_predictions.head()

## Combine and Plot Forecast Results

Visualizing the forecast results by plotting a bar chart comparing the historical and predicted consumption data for a specific item.

In [ ]:
import matplotlib.pyplot as plt

# Choose an item to plot (you can change this to any item in your dataset)
item_to_plot = items[0]  # This selects the first item in the list

# Filter data for the chosen item
historical_data = monthly_data[monthly_data['ITEM DESCRIPTION'] == item_to_plot]
predicted_data = final_predictions[final_predictions['Item'] == item_to_plot]

# Create the plot
plt.figure(figsize=(15, 7))

# Plot historical data
plt.plot(historical_data.index, historical_data['Consumption'], label='Historical', color='springgreen', marker='o')

# Plot predicted data
plt.plot(predicted_data['Date'], predicted_data['Predicted_Consumption'], label='Predicted', color='deepskyblue', marker='o')

# Add labels and title
plt.title(f'Consumption Pattern for {item_to_plot}')
plt.xlabel('Date')
plt.ylabel('Consumption')

# Add legend
plt.legend()

# Show grid
plt.grid(True, linestyle='--', alpha=0.7)


# Rotate x-axis labels for better readability
plt.xticks(rotation=45)

# Adjust layout to prevent cutting off labels
plt.tight_layout()

# Show the plot
plt.show()

# Print some information about the plot
print(f"Plotting consumption for: {item_to_plot}")
print(f"Historical data range: {historical_data.index.min()} to {historical_data.index.max()}")
print(f"Prediction range: {predicted_data['Date'].min()} to {predicted_data['Date'].max()}")